In [36]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from data import *

In [37]:
filepath = os.path.join("results")

# Find files sorted by time
files = sorted(glob.glob(os.path.join(filepath, "*.hdf5")), key=lambda f: os.stat(f).st_mtime)
print(files)

# Choose and open one file
filename = files[0]
try:
    f.close()
except:
    pass
f = h5py.File(filename, "r")

print(len(files))

['results/data-03-02_18-08.hdf5', 'results/data-03-02_18-09.hdf5', 'results/data-03-02_18-13.hdf5', 'results/data-03-02_18-14.hdf5', 'results/data-03-02_18-15.hdf5', 'results/data-03-02_18-16.hdf5', 'results/data-03-02_18-17.hdf5', 'results/data-03-02_18-18.hdf5', 'results/data-03-02_18-19.hdf5', 'results/data-03-02_18-20.hdf5', 'results/data-03-02_18-21.hdf5', 'results/data-03-02_18-22.hdf5', 'results/data-03-02_18-23.hdf5', 'results/data-03-02_18-24.hdf5', 'results/data-03-02_18-25.hdf5', 'results/data-03-02_18-26.hdf5', 'results/data-03-02_18-27.hdf5']
17


In [38]:
# Plot mean observation
mean_observation = f["mean_observation"][()]
plt.imshow(mean_observation[::-1,:])
plt.show()

KeyError: "Unable to open object (object 'mean_observation' doesn't exist)"

In [46]:
want=[0,3,10,14,len(files)-1]

In [93]:
# Iterate through data file
# Choose and open one file
col=['r','m','c', 'g','b',]
k=0
endpts=[]
fig, ax = plt.subplots(figsize=(16, 9))
for i,filename in enumerate(files):
    #filename = files[0]
    if i not in want:
        continue
    try:
        f.close()
    except:
        pass
    f = h5py.File(filename, "r")


    o  = []
    a  = []
    r  = []
    x  = []
    dx = []
    T  = 0
    size_batch = 10

    episodes = f["episodes"]
    for i in episodes:
        if T >= size_batch:
            break
        try:
            grp = episodes[i]
            actions      = grp["actions"][()]
            observations = grp["observations"][()]
            rewards      = grp["rewards"][()]
            xs  = grp["xs"][()]
            dxs = grp["dxs"][()]
        except:
            continue

        # Append data to lists
        o.append(observations)
        a.append(actions)
        r.append(rewards)
        x.append(xs[...,:2] - np.array([0, -0.45])[np.newaxis,:])
        dx.append(dxs[...,:2])
        T += rewards.shape[0]

        #print("Episode {}: {}".format(i, T))

    # Gather batch
    o  = np.concatenate(o, axis=0) # [T x 150 x 200 x 3]
    a  = np.concatenate(a, axis=0) # [T x 2]
    r  = np.concatenate(r, axis=0) # [T]
    x  = np.concatenate(x, axis=0) # [T x 2]
    dx = np.concatenate(dx, axis=0) # [T x 2]
    ax.plot(x[:,0], x[:,1], color=col[k])
    
    #append endpts for seixer plots
    endpts.append(x[-1,:])
    #plt.plot([x[0,0]], [x[0,1]], color=col[k], marker='o')
    #plt.plot([x[-1,0]], [x[-1,1]], color=col[k], marker='s')
    
    k+=1

labels=["itr: "+str(itr)   for itr in want ]

for i, c in enumerate(col):
    x=endpts[i]
    ax.plot([x[0]], [x[1]], color=col[i], marker='s', markersize=14)
ax.plot([0.], [0.], color='k', marker='o', markersize=14)

#goal region
ang=np.linspace(0.0, -np.pi/2, 100)
arcx=[-.45+np.cos(a)*.5 for a in ang]
arcy=[.55+np.sin(a)*.5 for a in ang]
#ax.plot(arcx, arcy, color='k')

ax.legend(labels, loc='best')
ax.axis('equal')
ax.axis('off')
plt.savefig('dqn_milestone.png')
#plt.show()





In [43]:

plt.show()

In [11]:
x.shape

(100, 2)